<a href="https://colab.research.google.com/github/AnirudhJM24/chatbot/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Embedding
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



In [2]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

df = pd.DataFrame()
df['text'] = training_sentences
df['labels'] = training_labels


In [3]:
df

,text,labels
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Hey! how are you?,greeting
5,Hi how are you doing?,greeting
6,Good morning,greeting
7,Good Afternoon,greeting
8,What's up,greeting
9,Bye,goodbye


In [4]:
one_hot = pd.get_dummies(df['labels'])


In [5]:
training_labels = one_hot.values

In [6]:
lbl = LabelEncoder()
lbls = lbl.fit_transform(df['labels'])

In [7]:
labels_inference = ['greeting','goodbye','thanks','about','name','help']

In [8]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [9]:
model = keras.Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(6, activation='softmax'))


In [10]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(padded_sequences,np.array(lbls),epochs = 500)

Epoch 1/500
2/2 [==============================] - 3s 14ms/step - loss: 1.7952 - accuracy: 0.0882
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 1.7936 - accuracy: 0.0882
Epoch 3/500
2/2 [==============================] - 0s 7ms/step - loss: 1.7929 - accuracy: 0.0882
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 1.7918 - accuracy: 0.0882
Epoch 5/500
2/2 [==============================] - 0s 8ms/step - loss: 1.7909 - accuracy: 0.0882
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 1.7895 - accuracy: 0.0882
Epoch 7/500
2/2 [==============================] - 0s 7ms/step - loss: 1.7882 - accuracy: 0.3235
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 1.7871 - accuracy: 0.2647
Epoch 9/500
2/2 [==============================] - 0s 7ms/step - loss: 1.7859 - accuracy: 0.2647
Epoch 10/500
2/2 [==============================] - 0s 7ms/step - loss: 1.7854 - accuracy: 0.2647
Epoch 11/500
2/2 [==========

In [12]:
def inference(model,sentence, tokenizer):
  sequence = tokenizer.texts_to_sequences(sentence)
  test = pad_sequences(sequence, maxlen=20)
  print(lbl.inverse_transform([np.argmax(model.predict(test))]))


In [13]:
inference(model, ['can u help me'],tokenizer)

['help']


In [14]:
model.save('chatbot.h5')

In [17]:
model1 = keras.Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model1.add(keras.layers.Bidirectional(keras.layers.LSTM(20,dropout=0.2)))
model1.add(keras.layers.Dense(6,activation='softmax'))


In [18]:
model1.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
model1.fit(padded_sequences,np.array(lbls),epochs = 500)

Epoch 1/500
2/2 [==============================] - 5s 19ms/step - loss: 1.7915 - accuracy: 0.2059
Epoch 2/500
2/2 [==============================] - 0s 10ms/step - loss: 1.7883 - accuracy: 0.2647
Epoch 3/500
2/2 [==============================] - 0s 10ms/step - loss: 1.7850 - accuracy: 0.2647
Epoch 4/500
2/2 [==============================] - 0s 11ms/step - loss: 1.7830 - accuracy: 0.2647
Epoch 5/500
2/2 [==============================] - 0s 11ms/step - loss: 1.7824 - accuracy: 0.2647
Epoch 6/500
2/2 [==============================] - 0s 11ms/step - loss: 1.7799 - accuracy: 0.2647
Epoch 7/500
2/2 [==============================] - 0s 10ms/step - loss: 1.7782 - accuracy: 0.2941
Epoch 8/500
2/2 [==============================] - 0s 11ms/step - loss: 1.7757 - accuracy: 0.3235
Epoch 9/500
2/2 [==============================] - 0s 11ms/step - loss: 1.7755 - accuracy: 0.2059
Epoch 10/500
2/2 [==============================] - 0s 12ms/step - loss: 1.7720 - accuracy: 0.2059
Epoch 11/500
2/2 [=

In [23]:
model1.save('chatbot1.h5')

In [22]:
inference(model1, ['ho are you'],tokenizer)

['about']


In [ ]:
import pickle

In [ ]:
with open('lblencoder.pkl','wb') as f:
  pickle.dump(lbl,f)
with open('tokenizer.pkl','wb') as f:
  pickle.dump(tokenizer,f)